In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
MLP 기반 3단계 이상 탐지 시스템 - ONNX 학습 스크립트

각 이상 유형(S1=fluctuation, S2=unbalance)에 대해 3단계 분류:
- 정상 (Normal)
- 황색 경보 (Yellow Alert)
- 적색 경보 (Red Alert)

출력: [S1_정상, S1_황색, S1_적색, S2_정상, S2_황색, S2_적색] (6개)
각 이상 유형별로 softmax를 적용하여 확률 분포 반환
"""

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import datetime

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# ONNX imports
import onnx
import onnxruntime as ort

# =====================================
# 설정
# =====================================
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.001

# 데이터 파일 경로
DATA_FILES = {
    'normal': 'data/processed/1124_features.csv',
    's1_yellow': 'data/processed/1120_features_S1_yellow.csv',
    's1_red': 'data/processed/1120_features_S1.csv',
    's2_yellow': 'data/processed/1120_features_S2_yellow.csv',
    's2_red': 'data/processed/1120_features_S2.csv',
}

# 레이블 매핑 (독립적인 3-way classification × 2)
# [S1_정상, S1_황색, S1_적색, S2_정상, S2_황색, S2_적색]
# S1과 S2가 독립적으로 학습되므로 각각 [정상, 황색, 적색] 중 하나만 활성화
LABEL_MAPPING = {
    'normal':    [1, 0, 0,  1, 0, 0],  # S1 정상, S2 정상
    's1_yellow': [0, 1, 0,  1, 0, 0],  # S1 황색, S2 정상
    's1_red':    [0, 0, 1,  1, 0, 0],  # S1 적색, S2 정상
    's2_yellow': [1, 0, 0,  0, 1, 0],  # S1 정상, S2 황색
    's2_red':    [1, 0, 0,  0, 0, 1],  # S1 정상, S2 적색
}

# 레이블 이름
ALERT_LEVELS = ['Normal', 'Yellow', 'Red']
ANOMALY_TYPES = ['S1_Fluctuation', 'S2_Unbalance']

# 출력 디렉토리 설정
OUTPUT_DIR_MODELS = 'models'
OUTPUT_DIR_RESULTS = 'results/mlp_3level'

# 디렉토리 생성
os.makedirs(OUTPUT_DIR_MODELS, exist_ok=True)
os.makedirs(OUTPUT_DIR_RESULTS, exist_ok=True)

# 재현성 설정
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)


# =====================================
# 1. PyTorch MLP 모델 정의
# =====================================
class MLPClassifier3Level(nn.Module):
    """
    3단계 경보 시스템을 위한 MLP 분류기
    
    독립적인 3-way classification × 2:
    - S1 (fluctuation): [정상, 황색, 적색] - softmax
    - S2 (unbalance): [정상, 황색, 적색] - softmax
    
    출력: 6개 노드 [S1_N, S1_Y, S1_R, S2_N, S2_Y, S2_R]
    """
    def __init__(self, input_size, hidden_sizes=(64, 32)):
        super(MLPClassifier3Level, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], 6)  # 6개 출력
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        # S1, S2 각각에 독립적으로 softmax 적용
        s1_logits = x[:, :3]  # [정상, 황색, 적색]
        s2_logits = x[:, 3:]  # [정상, 황색, 적색]
        
        s1_probs = self.softmax(s1_logits)
        s2_probs = self.softmax(s2_logits)
        
        # 결합
        output = torch.cat([s1_probs, s2_probs], dim=1)
        return output


# =====================================
# 2. 데이터 로드 및 레이블 생성
# =====================================
print("=" * 60)
print("📂 Loading Data from Multiple Scenarios")
print("=" * 60)

all_features = []
all_labels = []

for scenario, filepath in DATA_FILES.items():
    if not os.path.exists(filepath):
        print(f"⚠️  Warning: {filepath} not found, skipping {scenario}")
        continue
    
    # 특징 벡터 로드
    df = pd.read_csv(filepath)
    features = df.iloc[:, 3:].values  # window_id, start_time, end_time 제외
    
    # 레이블 생성
    label = np.array(LABEL_MAPPING[scenario])
    labels = np.tile(label, (len(features), 1))
    
    all_features.append(features)
    all_labels.append(labels)
    
    print(f"✅ Loaded {scenario:15s}: {features.shape[0]:4d} samples")

# 데이터 통합
X = np.vstack(all_features)
y = np.vstack(all_labels)

print(f"\n{'=' * 60}")
print(f"📊 Combined Dataset")
print(f"{'=' * 60}")
print(f"✅ Total samples: {X.shape[0]}")
print(f"✅ Feature shape: {X.shape}")
print(f"✅ Label shape: {y.shape}")

# 레이블 분포 확인
print(f"\n[Label Distribution]")
for scenario, label_vec in LABEL_MAPPING.items():
    count = np.sum(np.all(y == label_vec, axis=1))
    percentage = count / len(y) * 100
    print(f"   {scenario:15s}: {count:4d} samples ({percentage:5.1f}%)")


# =====================================
# 3. 데이터 전처리 및 분할
# =====================================
print(f"\n{'=' * 60}")
print(f"🔧 Data Preprocessing")
print(f"{'=' * 60}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# stratify를 위한 레이블 변환 (6-dim → single label)
def label_to_single(y_multi):
    """6-dim one-hot을 단일 레이블로 변환"""
    labels = []
    for row in y_multi:
        if np.array_equal(row, [1,0,0,1,0,0]):
            labels.append(0)  # normal
        elif np.array_equal(row, [0,1,0,1,0,0]):
            labels.append(1)  # s1_yellow
        elif np.array_equal(row, [0,0,1,1,0,0]):
            labels.append(2)  # s1_red
        elif np.array_equal(row, [1,0,0,0,1,0]):
            labels.append(3)  # s2_yellow
        elif np.array_equal(row, [1,0,0,0,0,1]):
            labels.append(4)  # s2_red
        else:
            labels.append(5)  # unknown
    return np.array(labels)

y_stratify = label_to_single(y)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.15, random_state=RANDOM_SEED, stratify=y_stratify
)

print(f"✅ Train samples: {X_train.shape[0]}")
print(f"✅ Test samples: {X_test.shape[0]}")


# =====================================
# 4. 스케일러 저장
# =====================================
scaler_path = os.path.join(OUTPUT_DIR_MODELS, 'scaler_mlp_3level.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved: {scaler_path}")

# 훈련/테스트 데이터 저장
data_split_path = os.path.join(OUTPUT_DIR_MODELS, 'data_split_3level.npz')
np.savez(data_split_path, 
         X_train=X_train, X_test=X_test, 
         y_train=y_train, y_test=y_test)
print(f"✅ Data split saved: {data_split_path}")


# =====================================
# 5. PyTorch 데이터 준비
# =====================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Using device: {device}")

# NumPy to Tensor
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# =====================================
# 6. 모델 초기화
# =====================================
input_size = X_train.shape[1]
model = MLPClassifier3Level(input_size=input_size, hidden_sizes=(64, 32))
model = model.to(device)

# 손실 함수: CrossEntropyLoss를 S1, S2 각각에 적용
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\n{'=' * 60}")
print(f"🏗️  Model Architecture")
print(f"{'=' * 60}")
print(f"   Input size: {input_size}")
print(f"   Architecture: {input_size} -> 64 -> 32 -> 6")
print(f"   Output: [S1_normal, S1_yellow, S1_red, S2_normal, S2_yellow, S2_red]")
print(f"   Total parameters: {sum(p.numel() for p in model.parameters())}")


# =====================================
# 7. 모델 학습
# =====================================
print(f"\n{'=' * 60}")
print("🚀 Training MLP Classifier (3-Level Alert System)")
print(f"{'=' * 60}")

loss_history = []
start_time = datetime.now()

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        
        # S1, S2 각각의 레이블 추출
        s1_labels = torch.argmax(batch_y[:, :3], dim=1)  # [0, 1, 2]
        s2_labels = torch.argmax(batch_y[:, 3:], dim=1)  # [0, 1, 2]
        
        # S1, S2 출력 추출
        s1_outputs = outputs[:, :3]
        s2_outputs = outputs[:, 3:]
        
        # 손실 계산 (두 개의 CrossEntropyLoss 합산)
        loss_s1 = criterion(s1_outputs, s1_labels)
        loss_s2 = criterion(s2_outputs, s2_labels)
        loss = loss_s1 + loss_s2
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    loss_history.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1:2d}/{EPOCHS}], Loss: {avg_loss:.4f}")

training_time = (datetime.now() - start_time).total_seconds()
print(f"\n✅ Training completed in {training_time:.2f} seconds")


# =====================================
# 8. ONNX 모델로 변환
# =====================================
print(f"\n{'=' * 60}")
print("💾 Converting to ONNX Format")
print(f"{'=' * 60}")

# 모델을 평가 모드로 전환
model.eval()
model = model.cpu()

# 더미 입력 생성
dummy_input = torch.randn(1, input_size, requires_grad=True)

# ONNX 파일 경로
onnx_path = os.path.join(OUTPUT_DIR_MODELS, 'mlp_classifier_3level.onnx')

# ONNX로 변환
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)
print(f"✅ ONNX model saved: {onnx_path}")

# ONNX 모델 검증
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print(f"✅ ONNX model validation passed")


# =====================================
# 9. ONNX Runtime으로 모델 테스트
# =====================================
print(f"\n{'=' * 60}")
print("🔍 Testing ONNX Model with ONNX Runtime")
print(f"{'=' * 60}")

# ONNX Runtime 세션 생성
ort_session = ort.InferenceSession(onnx_path)

# 테스트 데이터로 추론
test_input = X_test[:5].astype(np.float32)
ort_inputs = {ort_session.get_inputs()[0].name: test_input}
ort_outputs = ort_session.run(None, ort_inputs)

print(f"✅ ONNX Runtime inference test successful")
print(f"   Input shape: {test_input.shape}")
print(f"   Output shape: {ort_outputs[0].shape}")
print(f"\n   Sample predictions (first 5):")
for i in range(5):
    pred = ort_outputs[0][i]
    s1_probs = pred[:3]
    s2_probs = pred[3:]
    print(f"   [{i}] S1: [N={s1_probs[0]:.3f}, Y={s1_probs[1]:.3f}, R={s1_probs[2]:.3f}], "
          f"S2: [N={s2_probs[0]:.3f}, Y={s2_probs[1]:.3f}, R={s2_probs[2]:.3f}]")

# PyTorch vs ONNX 결과 비교
model.eval()
with torch.no_grad():
    torch_input = torch.FloatTensor(test_input)
    torch_output = model(torch_input).numpy()

diff = np.abs(torch_output - ort_outputs[0])
print(f"\n✅ PyTorch vs ONNX difference: max={diff.max():.6f}, mean={diff.mean():.6f}")


# =====================================
# 10. PyTorch 모델도 저장
# =====================================
model_path = os.path.join(OUTPUT_DIR_MODELS, 'mlp_classifier_3level.pth')
torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': input_size,
    'hidden_sizes': (64, 32),
    'output_size': 6,
    'training_info': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'training_time': training_time,
        'final_loss': loss_history[-1],
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
}, model_path)
print(f"✅ PyTorch model saved: {model_path}")


# =====================================
# 11. Loss Curve 시각화
# =====================================
print(f"\n{'=' * 60}")
print("📊 Generating Visualizations")
print(f"{'=' * 60}")

plt.figure(figsize=(10, 6))
plt.plot(loss_history, color='royalblue', linewidth=2, marker='o', markersize=4)
plt.title("MLP Training Loss Curve (3-Level Alert System)", fontsize=14, fontweight='bold')
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("Loss (CrossEntropy)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()

loss_curve_path = os.path.join(OUTPUT_DIR_RESULTS, 'mlp_3level_loss_curve.png')
plt.savefig(loss_curve_path, dpi=300, bbox_inches='tight')
print(f"✅ Loss curve saved: {loss_curve_path}")
plt.close()


# =====================================
# 12. 훈련 정보 저장
# =====================================
training_summary = {
    'model_type': 'MLP Classifier (3-Level Alert System) - ONNX',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'data_sources': DATA_FILES,
    'label_mapping': {k: v for k, v in LABEL_MAPPING.items()},
    'dataset': {
        'total_samples': len(X),
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'feature_dim': input_size,
        'label_distribution': {
            scenario: int(np.sum(np.all(y == label_vec, axis=1)))
            for scenario, label_vec in LABEL_MAPPING.items()
        }
    },
    'architecture': {
        'input_size': input_size,
        'hidden_layers': [64, 32],
        'output_size': 6,
        'output_structure': '[S1_normal, S1_yellow, S1_red, S2_normal, S2_yellow, S2_red]',
        'total_parameters': sum(p.numel() for p in model.parameters())
    },
    'hyperparameters': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'optimizer': 'Adam',
        'loss_function': 'CrossEntropyLoss (S1 + S2)',
        'random_seed': RANDOM_SEED
    },
    'training_results': {
        'training_time_seconds': training_time,
        'final_loss': float(loss_history[-1]),
        'min_loss': float(min(loss_history)),
        'loss_history': [float(l) for l in loss_history]
    },
    'onnx_export': {
        'opset_version': 11,
        'input_name': 'input',
        'output_name': 'output',
        'dynamic_axes': True,
        'validation_passed': True
    },
    'saved_files': {
        'onnx_model': onnx_path,
        'pytorch_model': model_path,
        'scaler': scaler_path,
        'data_split': data_split_path,
        'loss_curve': loss_curve_path
    }
}

summary_path = os.path.join(OUTPUT_DIR_MODELS, 'training_summary_3level.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(training_summary, f, indent=4, ensure_ascii=False)
print(f"✅ Training summary saved: {summary_path}")


# =====================================
# 결과 요약 출력
# =====================================
print(f"\n{'=' * 60}")
print("✅ Training Completed Successfully")
print(f"{'=' * 60}")
print(f"\n[Model Info]")
print(f"  • System: 3-Level Alert (Normal, Yellow, Red)")
print(f"  • Anomaly Types: S1 (Fluctuation), S2 (Unbalance)")
print(f"  • Output Format: [S1_N, S1_Y, S1_R, S2_N, S2_Y, S2_R]")
print(f"  • Input size: {input_size}")
print(f"  • Architecture: {input_size} -> 64 -> 32 -> 6")
print(f"  • Training time: {training_time:.2f}s")
print(f"  • Final loss: {loss_history[-1]:.4f}")

print(f"\n[Saved Models]")
print(f"  • ONNX model: {onnx_path}")
print(f"  • PyTorch model: {model_path}")
print(f"  • Scaler: {scaler_path}")
print(f"  • Data split: {data_split_path}")

print(f"\n[Training Summary]")
print(f"  • Training summary: {summary_path}")
print(f"  • Loss curve: {loss_curve_path}")

print(f"\n[Next Steps]")
print(f"  Use 'predict_onnx_3level.py' to run inference:")
print(f"    python predict_onnx_3level.py --csv your_data.csv")
print(f"{'=' * 60}\n")

📂 Loading Data from Multiple Scenarios
✅ Loaded normal         :  723 samples
✅ Loaded s1_yellow      :  734 samples
✅ Loaded s1_red         :  741 samples
✅ Loaded s2_yellow      :  736 samples
✅ Loaded s2_red         :  757 samples

📊 Combined Dataset
✅ Total samples: 3691
✅ Feature shape: (3691, 15)
✅ Label shape: (3691, 6)

[Label Distribution]
   normal         :  723 samples ( 19.6%)
   s1_yellow      :  734 samples ( 19.9%)
   s1_red         :  741 samples ( 20.1%)
   s2_yellow      :  736 samples ( 19.9%)
   s2_red         :  757 samples ( 20.5%)

🔧 Data Preprocessing
✅ Train samples: 3137
✅ Test samples: 554
✅ Scaler saved: models\scaler_mlp_3level.pkl
✅ Data split saved: models\data_split_3level.npz

✅ Using device: cpu

🏗️  Model Architecture
   Input size: 15
   Architecture: 15 -> 64 -> 32 -> 6
   Output: [S1_normal, S1_yellow, S1_red, S2_normal, S2_yellow, S2_red]
   Total parameters: 3302

🚀 Training MLP Classifier (3-Level Alert System)
Epoch [ 5/30], Loss: 1.2521
Epoch

C:\Users\AICA\AppData\Local\Temp\ipykernel_14916\3919820276.py:325: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1125 11:05:15.157000 14916 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W1125 11:05:15.961000 14916 site-packages\torch\onnx\_internal\exporter\_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `MLPClassifier3Level([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `MLPClassifier3Level([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 11).
Failed to convert the model to the target version 11 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "C:\Users\AICA\anaconda3\Lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
        func=_partial_convert_version, model=model
    )
  File "C:\Users\AICA\anaconda3\Lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
  File "C:\Users\AICA\anaconda3\Lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        proto, target_version=self.target_version
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 2 of general pattern rewrite rules.
✅ ONNX model saved: models\mlp_classifier_3level.onnx
✅ ONNX model validation passed

🔍 Testing ONNX Model with ONNX Runtime
✅ ONNX Runtime inference test successful
   Input shape: (5, 15)
   Output shape: (5, 6)

   Sample predictions (first 5):
   [0] S1: [N=0.000, Y=0.000, R=1.000], S2: [N=1.000, Y=0.000, R=0.000]
   [1] S1: [N=1.000, Y=0.000, R=0.000], S2: [N=1.000, Y=0.000, R=0.000]
   [2] S1: [N=1.000, Y=0.000, R=0.000], S2: [N=0.000, Y=0.998, R=0.002]
   [3] S1: [N=1.000, Y=0.000, R=0.000], S2: [N=1.000, Y=0.000, R=0.000]
   [4] S1: [N=0.000, Y=1.000, R=0.000], S2: [N=1.000, Y=0.000, R=0.000]

✅ PyTorch vs ONNX difference: max=0.000000, mean=0.000000
✅ PyTorch model saved: models\mlp_classifier_3level.pth

📊 Generating Visualizations
✅ Loss curve saved: results/mlp_3level\mlp_3level_loss_curve.png
✅ Trainin